In [1]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
from datetime import datetime

In [2]:
df = pd.read_csv("reestr_17_1831.csv",encoding='utf-8',sep=';')

In [3]:
try:
    del df['интерв_ср_буд']
    del df['интерв_ср_вых']
    del df['интерв_ср_вчр']
    print(1)
except:
    print(0)
    pass

1


## Режим работы

In [4]:
def fnd_tm_strt_end(string):
    
    lst_dgt = re.findall(r'\d+', string)
    if len(lst_dgt) > 1:
        
        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        l1.sort()
        tm_strt = min(l1)
        tm_end = max(l1)
        cnt=0
        while ((tm_strt < 5) & (tm_strt < tm_end)):
            try:
                tm_strt = min(l1[cnt:])
            except:
                tm_strt = 6
                break
            cnt+=1
        #
        cnt=len(l1)
        while ((tm_end > 23) & (tm_strt < tm_end)):
            try:
                tm_end = max(l1[:cnt])
            except:
                tm_end = 22
                break
            cnt-=1
        #
    #
    else:
        tm_strt = 6
        tm_end = 22
    #
    return tm_strt, tm_end

In [96]:
lst_reg_rab = list(df.режим_работы)

lst_strt_wrk,lst_end_wrk,lst_strt_hld,lst_end_hld = [],[],[],[]

i=0
for i in range(len(lst_reg_rab)):
    string = str(lst_reg_rab[i])
    
    lst_wrd = ['январ','феврал','март','апрел','май','мая','мае',
                   'июн','июл','август','сентябр','октябр','ноябр', 'декабр']
    lst_days = ['понедельник','вторник','сред',
            'четверг','пятниц','суббот','вокресен']
    if string == 'nan':
        tm_strt_wrk, tm_end_wrk = '-','-'
        tm_strt_hld, tm_end_hld = '-','-'
    #
    elif 'жедневно' in string:
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(string)
        tm_strt_hld, tm_end_hld = tm_strt_wrk, tm_end_wrk
    #
    elif 'рабочи' in string:
        ind_hld = string.find('выходны')
        str_wrk = string[:ind_hld]
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(str_wrk)
        if 'выходны' in string:
            str_hld = string[ind_hld:]
            tm_strt_hld, tm_end_hld = fnd_tm_strt_end(str_hld)
        else:
            tm_strt_hld, tm_end_hld = '-','-'
    #
    elif 'выходны' in string:
        if 'рабочи' in string:
            continue
        elif (any(prt in string for prt in lst_wrd)):
            ind_hld = string.find('выходны')
            string2 = string[:ind_hld]
            string3 = string[ind_hld:]
            tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(string2)
            tm_strt_hld, tm_end_hld = fnd_tm_strt_end(string3)
        elif (any(prt in string for prt in lst_days)):
            tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(string)
            tm_strt_hld, tm_end_hld = tm_strt_wrk, tm_end_wrk
        else:
            tm_strt_wrk, tm_end_wrk = '-','-'
            tm_strt_hld, tm_end_hld = fnd_tm_strt_end(string)
    #
    else:
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(string)
        tm_strt_hld, tm_end_hld = tm_strt_wrk, tm_end_wrk
    #
    def mk_tm(one):
        if one != '-':
            one = str(one) + ':00'
        return one
    # 
    lst_strt_wrk.append(mk_tm(tm_strt_wrk))
    lst_end_wrk.append(mk_tm(tm_end_wrk))
    lst_strt_hld.append(mk_tm(tm_strt_hld))
    lst_end_hld.append(mk_tm(tm_end_hld))
# 


In [6]:
# data1=list(zip(lst_reg_rab,lst_strt_wrk,lst_end_wrk,lst_strt_hld,lst_end_hld))
# columns1=['string','wrk_strt','wrk_end','hld_strt','hld_end']

# tmp_df = pd.DataFrame(data=data1,columns=columns1)

## Интервал

In [7]:
def fnd_interv(string):
    ind_sep = string.find(':')
    if ind_sep == -1:
        ind_sep = 0
    #
    ind_mnt = string.find('мин')
    str_part = string[ind_sep:ind_mnt]

    lst_dgt = re.findall(r'\d+', str_part)
    if len(lst_dgt) > 0:

        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        l1.sort()
        min_interv = min(l1)
        max_interv = max(l1)
    #
    else:
        min_interv = 1
        max_interv = 180
    #

    return min_interv, max_interv

In [8]:
def fnd_day_tp(string,tp,rplc_min,replc_max):
    if tp in string:
        ind_tp = string.find(tp)
        if tp == 'мин':
            str_tp = string[:ind_tp]
        else:
            str_tp = string[ind_tp:]
        min_res,max_res = fnd_interv(str_tp)
    else:
        min_res,max_res = rplc_min,replc_max
    #
    return min_res,max_res

In [ ]:
def find_max_pair(string):

    lst_prds = []
    lst_mnt = re.findall(' мин', string)
    str1 = string
    i=0
    for i in range(len(lst_mnt)):

        ind_mnt = str1.find(' мин')
        ind_sep = str1.find(':')
        if ind_sep == -1:
            ind_sep = 0
        str2 = str1[ind_sep:ind_mnt]
        lst_dgt = re.findall(r'\d+', str2)
        lst_new_dgt=[]
        if len(lst_dgt) > 0:
            j=0
            for j in range(len(lst_dgt)):
                dgt = int(lst_dgt[j])
                lst_new_dgt.append(dgt)
        else:
            lst_new_dgt.append(0)
        #

        str1 = str1[ind_mnt:].replace(' мин','',1)

        lst_prds.append(lst_new_dgt)
    #
    k=0
    tm_min,tm_max = 0,0
    for k in range(len(lst_prds)):
        one_lst = lst_prds[k]
        if ((tm_max <= max(one_lst)) & (tm_min <= min(one_lst))):
            tm_min = min(one_lst)
            tm_max = max(one_lst)
        #
    #
    return tm_min,tm_max

In [50]:
lst_interv = list(df.интервалы)

lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max = [],[],[],[]
lst_evn_wrk_min,lst_evn_wrk_max,lst_evn_hld_min,lst_evn_hld_max = [],[],[],[]

i=0
for i in range(len(lst_interv)):
    string=str(lst_interv[i])
    if ((string == 'nan') | ('расписани' in string)):
        wrk_min,wrk_max,hld_min,hld_max = '-','-','-','-'
        evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max = '-','-','-','-'
    elif (('рабочи' in string) | ('выходн' in string) | ('вечерн' in string)):
        wrk_min,wrk_max = fnd_day_tp(string,'рабочи',0,0)
        if ((wrk_min==0)&(wrk_max == 0)):
            wrk_min,wrk_max = fnd_day_tp(string,'мин',0,0)
        #
        hld_min,hld_max = fnd_day_tp(string,'выходн',wrk_min,wrk_max)
        evn_wrk_min,evn_wrk_max = fnd_day_tp(string,'вечерн',wrk_min,wrk_max)
    #
    elif 'мин' in string:
        lst_wrd = ['январ','феврал','март','апрел','май','мая','мае',
                   'июн','июл','август','сентябр','октябр','ноябр', 'декабр']
        if any(prt in string for prt in lst_wrd):
            wrk_min,wrk_max = find_max_pair(string)
        else:
            wrk_min,wrk_max = fnd_interv(string)
        hld_min,hld_max,evn_wrk_min,evn_wrk_max = wrk_min,wrk_max,wrk_min,wrk_max
    else:
        wrk_min,wrk_max,hld_min,hld_max = 0,0,0,0
        evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max = 0,0,0,0
    #
    evn_hld_min,evn_hld_max = evn_wrk_min,evn_wrk_max
    
    lst_value=[wrk_min,wrk_max,hld_min,hld_max,
               evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max]
    lst_lsts=[lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max,
             lst_evn_wrk_min,lst_evn_wrk_max,lst_evn_hld_min,lst_evn_hld_max]
    j=0
    for j in range(len(lst_value)):
        lst_lsts[j].append(lst_value[j])
#

In [10]:
# data2 = list(zip('wrk_min','wrk_max','hld_min','hld_max',
#                  'evn_wrk_min','evn_wrk_max','evn_hld_min','evn_hld_max'))

## Выпуск

In [11]:
def fnd_vpsk(string,tp):
    if tp == 'машин':
        ind_strt = 0
    elif tp == 'вагон':
        ind_strt = 0
    else:
        ind_strt = string.find(tp)
        if ind_strt == -1:
            ind_strt = 0
            return 0
    #
    str_prt = string[ind_strt:]
    ind_veh = str_prt.find('машин')
    if ind_veh == -1:
        ind_veh = str_prt.find('вагон')
    str_tp = str_prt[:ind_veh]

    lst_dgt = re.findall(r'\d+', str_tp)
    if len(lst_dgt) > 0:
        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        vpsk = max(l1)
    else:
        vpsk = 0
    #
    return vpsk

In [59]:
lst_vpsk = list(df.выпуск)

lst_vpsk_wrk,lst_vpsk_sbt,lst_vpsk_vsk = [],[],[]

i=0
for i in range(len(lst_vpsk)):
    string=str(lst_vpsk[i])
    if ((string == 'nan')):
        vpsk_wrk,vpsk_sbt,vpsk_vsk = '-','-','-'
        
    elif (('рабочи' in string) | ('выходн' in string) 
          | ('суббот' in string) | ('воскресен' in string)):
        vpsk_wrk = fnd_vpsk(string,'рабочим')
        if vpsk_wrk == 0:
            vpsk_wrk = fnd_vpsk(string,'машин')
        vpsk_sbt = fnd_vpsk(string,'суббот')
        vpsk_vsk = fnd_vpsk(string,'воскресен')
        if ((vpsk_sbt == 0) | (vpsk_vsk == 0)):
            if vpsk_sbt == vpsk_vsk:
                vpsk_sbt = fnd_vpsk(string,'выходн')
                vpsk_vsk = vpsk_sbt
            else:
                if vpsk_vsk > 0:
                    vpsk_sbt = vpsk_vsk
                else:
                    vpsk_vsk = vpsk_sbt
        #
    #
    elif 'машин' in string:
        vpsk_wrk = fnd_vpsk(string,'машин')
        vpsk_sbt,vpsk_vsk = vpsk_wrk,vpsk_wrk
    elif 'вагон' in string:
        vpsk_wrk = fnd_vpsk(string,'вагон')
        vpsk_sbt,vpsk_vsk = vpsk_wrk,vpsk_wrk
    else:
        vpsk_wrk,vpsk_sbt,vpsk_vsk = 0,0,0
    #
    lst_vpsk_wrk.append(vpsk_wrk)
    lst_vpsk_sbt.append(vpsk_sbt)
    lst_vpsk_vsk.append(vpsk_vsk)
    
#

In [13]:
# data3 = list(zip(lst_vpsk_wrk,lst_vpsk_sbt,lst_vpsk_vsk))

## Время в пути

In [14]:
def fnd_dgt(string):
    if '66-го' in string:
        string = string.replace('66-го','')
    lst_dgt = re.findall(r'\d+', string)
    if len(lst_dgt) > 0:
        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        tm_rt = max(l1)
    else:
        tm_rt = 0
    #
    return tm_rt

In [15]:
def get_hr_to_mnt(string):
    try:
        lst_splt = string.split('ч.')
        lst_dgt_hr = re.findall(r'\d+', lst_splt[0])
        hr = int(lst_dgt_hr[-1])

        lst_dgt_mnt = re.findall(r'\d+', lst_splt[1])
        mnt = int(lst_dgt_mnt[0])
        ttl_mnt = hr*60 + mnt
    except:
        
        try:
            lst_dgt_hr = re.findall(r'\d+', string)
            ttl_mnt = int(max(lst_dgt_hr))
        except:
            ttl_mnt = 0

    return ttl_mnt

In [16]:
lst_tm_rt_all = list(df.время_в_пути)

lst_tm_rt=[]

i=0
for i in range(len(lst_tm_rt_all)):
    string = str(lst_tm_rt_all[i])
    if string == 'nan':
        tm_rt = '-'
    #
    elif ('ч.' in string):
        string = string.replace('ч:','ч ')
        lst_hr = re.findall('мин', string)
        if len(lst_hr) == 1:
            tm_rt = get_hr_to_mnt(string)
        else:
            lst_prts=[]
            j=0
            for j in range(len(lst_hr)):
                ind_prt = string.find('мин')
                string1 = string[:ind_prt]
                string = string[ind_prt:].replace('мин','',1)
                try:
                    tm_rt1 = get_hr_to_mnt(string1)
                except:
                    print(i)
                    tm_rt1=0
                lst_prts.append(tm_rt1)
            # 
            tm_rt = max(lst_prts)

    elif 'мин' in string:
        tm_rt = fnd_dgt(string)
    else:
        tm_rt = 1111
    #
    lst_tm_rt.append(tm_rt)
# 

## Протяженность

In [17]:
lst_lngth_all = list(df.протяженность)

lst_lngth = []

i=0
for i in range(len(lst_lngth_all)):
    string = str(lst_lngth_all[i])
    if string == 'nan':
        lngth = '-'
    else:
        string = string.replace(',','.')
        lst_dgt_hr = re.findall("\d+\.\d+", string)
        if len(lst_dgt_hr) == 0:
            lst_dgt_hr = re.findall("\d+", string)
        #
        lst_dgt_flt=[]
        k=0
        for k in range(len(lst_dgt_hr)):
            dgt = float(lst_dgt_hr[k])
            lst_dgt_flt.append(dgt)
        #
        try:
            lngth = max(lst_dgt_flt)
        except:
            lngth='-'
    lst_lngth.append(lngth)
# 

## Подвижной состав

In [112]:
lst1 = list(df.подв_состав.unique())
lst_podv_sost_all = list(df.подв_состав)

In [113]:
models = pd.read_csv("models_ts.csv",encoding='windows-1251',sep=';')
lst_models=list(models.model)
lst_occup=list(models.occupancy)

In [114]:
set(lst_occup)

{'БК', 'МК', 'СК'}

In [123]:
lst_s_occp_s,lst_s_occp_m,lst_s_occp_b = [],[],[]

i=0
for i in range(len(lst_podv_sost_all)):
    string = str(lst_podv_sost_all[i])
    res_occp_s,res_occp_m,res_occp_b = '-','-','-'
    k=0
    for k in range(len(lst_models)):
        if lst_models[k] in string:
            occp = lst_occup[k]
            if occp == 'МК':
                res_occp_s = 1
            if occp == 'СК':
                res_occp_m = 1
            if occp == 'БК':
                res_occp_b = 1
    #
    lst_s_occp_s.append(res_occp_s)
    lst_s_occp_m.append(res_occp_m)
    lst_s_occp_b.append(res_occp_b)
# 
#         if any(mdl in string for mdl in lst_models)

In [ ]:
lst_s_occp_s,lst_s_occp_m,lst_s_occp_b

## Стоимость проезда

In [19]:
lst_money_all = list(df.стоимость_проезда)

lst_money = []

i=0
for i in range(len(lst_money_all)):
    string = str(lst_money_all[i])
    if string == 'nan':
        money = '-'
    else:
        string = string.replace(',','.')
        lst_dgt_hr = re.findall("\d+", string)
        #
        lst_dgt_flt=[]
        k=0
        for k in range(len(lst_dgt_hr)):
            dgt = int(lst_dgt_hr[k])
            lst_dgt_flt.append(dgt)
        #
        try:
            lst_dgt_flt.sort()
            money = max(lst_dgt_flt)
            if money >= 1000:
                cnt=-1
                while money >= 1000:
                    money = max(lst_dgt_flt[:cnt])
                    cnt-=1
                #
        except:
            money='-'
    lst_money.append(money)
#

# Вечернее время - начало периода

In [20]:
def fnd_evn_tm(string):
    ind_str = string.find('вечер')
    ind_end = string[ind_str:].find(')')
    string1 = string[ind_str:][:ind_end]
    lst_dgt = re.findall("\d+",string1)

    lst_dgt_flt=[]
    k=0
    for k in range(len(lst_dgt)):
        dgt = int(lst_dgt[k])
        lst_dgt_flt.append(dgt)
    #

    try:
        evn_tm = max(lst_dgt_flt)
    except:
        evn_tm = 0

    return evn_tm

In [21]:
lst_evn_tm_all=list(df.интервалы)
lst_evn_tm = []

for i in range(len(lst_evn_tm_all)):
    string = str(lst_evn_tm_all[i])
    if 'вечер' in string:
        evn_tm = fnd_evn_tm(string)
    else:
        evn_tm = '-'
    #
    lst_evn_tm.append(evn_tm)
# 

In [107]:
models = pd.read_csv("models_ts.csv",encoding='windows-1251',sep=';')

Index(['model', 'occupancy'], dtype='object')

# ________________________

In [ ]:
# # режим работы - 4
# lst_strt_wrk,lst_end_wrk,lst_strt_hld,lst_end_hld
# # интевал - 8
# lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max,lst_evn_wrk_min,lst_evn_wrk_max,lst_evn_hld_min,lst_evn_hld_max
# интервал - вечернее время, начало периода
# lst_evn_tm
# # выпуск - 3
# lst_vpsk_wrk,lst_vpsk_sbt,lst_vpsk_vsk
# # время в пути - 1
# lst_tm_rt
# # протяженность - 1
# lst_lngth
# # подвижной состав - 0
# # стоимость проезда - 1
# lst_money

In [125]:
lst_lsts=[lst_strt_wrk,lst_end_wrk,lst_strt_hld,lst_end_hld,
         lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max,
          lst_evn_wrk_min,lst_evn_wrk_max,
          lst_evn_tm,
         lst_vpsk_wrk,lst_vpsk_sbt,lst_vpsk_vsk,
          lst_s_occp_s,lst_s_occp_m,lst_s_occp_b,
         lst_tm_rt,lst_lngth,lst_money]

In [126]:
lst_new_clmns=['new_реж_раб_начл', 'new_реж_раб_конц',
         'new_реж_вых_начл', 'new_реж_вых_конц',
         'new_интерв_раб_мин', 'new_интерв_раб_макс',
         'new_интерв_вых_мин', 'new_интерв_вых_макс',
         'new_интерв_веч_мин', 'new_интерв_веч_макс','new_начало_веч_интерв_час',
         'new_выпуск_раб', 'new_выпуск_суб', 'new_выпуск_вск',
               'new_вмест_мк','new_вмест_ск','new_вмест_бк',
         'new_врем_пути', 'new_протяж_маршр', 'new_стоим_проезд']

In [127]:
print(len(lst_new_clmns))
print(len(lst_lsts))

20
20


In [128]:
df_new = df.copy()

In [129]:
i=0
for i in range(len(lst_new_clmns)):
    name = lst_new_clmns[i]
    clmn = lst_lsts[i]
    try:
        df_new[name] = clmn
    except:
        print(name)
# 

In [130]:
df_new.columns

Index(['населенный_пункт', 'тип_тс', 'тип_маршрута', 'номер_маршрута',
       'наим_маршрута', 'маршут_след_прям', 'маршут_след_обрат',
       'список_оп_прям', 'список_оп_обрат', 'режим_работы', 'интервалы',
       'выпуск', 'время_в_пути', 'протяженность', 'предприятие', 'подв_состав',
       'стоимость_проезда', 'место_отстоя', 'место_отправл', 'время_отправл',
       'пнд', 'втр', 'срд', 'чтв', 'птн', 'сбт', 'вск', 'ссылка',
       'new_реж_раб_начл', 'new_реж_раб_конц', 'new_реж_вых_начл',
       'new_реж_вых_конц', 'new_интерв_раб_мин', 'new_интерв_раб_макс',
       'new_интерв_вых_мин', 'new_интерв_вых_макс', 'new_интерв_веч_мин',
       'new_интерв_веч_макс', 'new_начало_веч_интерв_час', 'new_выпуск_раб',
       'new_выпуск_суб', 'new_выпуск_вск', 'new_вмест_мк', 'new_вмест_ск',
       'new_вмест_бк', 'new_врем_пути', 'new_протяж_маршр',
       'new_стоим_проезд'],
      dtype='object')

In [131]:
df_new2 = df_new[['населенный_пункт', 'тип_тс', 'тип_маршрута', 'номер_маршрута',
                  'наим_маршрута', 'маршут_след_прям', 'маршут_след_обрат',
                  'список_оп_прям', 'список_оп_обрат', 
                  'режим_работы',
                 'new_реж_раб_начл','new_реж_раб_конц', 
                  'new_реж_вых_начл', 'new_реж_вых_конц',
                 'интервалы',
                 'new_интерв_раб_мин', 'new_интерв_раб_макс', 
                  'new_интерв_вых_мин','new_интерв_вых_макс', 
                  'new_интерв_веч_мин','new_интерв_веч_макс', 
                  'new_начало_веч_интерв_час',
                 'выпуск',
                 'new_выпуск_раб', 'new_выпуск_суб','new_выпуск_вск',
                 'время_в_пути',
                 'new_врем_пути',
                 'протяженность',
                 'new_протяж_маршр',
                 'предприятие', 'подв_состав',
                  'new_вмест_мк','new_вмест_ск','new_вмест_бк',
                 'стоимость_проезда',
                 'new_стоим_проезд',
                  'место_отстоя', 'место_отправл', 'время_отправл',
                  'пнд', 'втр', 'срд','чтв', 'птн', 'сбт', 'вск', 'ссылка']]

In [132]:
df_new2.to_csv("reestr_17_1831_new_7.csv",encoding='windows-1251',sep=';',index=False)